# Quantization using the Model Compression Toolkit - example in Pytorch

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/add_tf_notebook/tutorials/keras_notebook/keras_notebook.ipynb)

## Overview

This quick start guide covers how to use the Model Compression Toolkit (MCT) for quantizing a PyTorch model. We will do so by giving an end-to-end example, training a model from scratch on MNIST data, then quantizing it using the MCT. Let us start with the relevant imports:

## Summary

In this tutorial we will cover:
1. Training a Pytorch model from scratch on MNIST.
2. Quantizing the model in a hardware-friendly manner (symmetric quantization, power-of-2 thresholds) using 8-bit activations and weights.
3. We will examine the output quantized model, evaluate it and compare its performance to the original model.
4. We will approximate the compression gains due to quantization.

## Setup

Install the relevant packages:

In [1]:
! pip install model-compression-toolkit
! pip install torch 
! pip install torchvision

Looking in indexes: https://pypi.org/simple, http://piprep-ub18:8080/simple
     |████████████████████████████████| 339 kB 909 kB/s eta 0:00:01


Looking in indexes: https://pypi.org/simple, http://piprep-ub18:8080/simple
Looking in indexes: https://pypi.org/simple, http://piprep-ub18:8080/simple


In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import model_compression_toolkit as mct

DepthwiseConv2D is defined in hardware model, but is not used in framework hardware model.


DepthwiseConv2D is defined in hardware model, but is not used in framework hardware model.


## Train a Pytorch classifier model on MNIST

Let us define the network and some helper functions to train and evaluate the model. These are taken from the official Pytorch examples https://github.com/pytorch/examples/blob/main/mnist/main.py

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

batch_size = 64
test_batch_size = 1000
random_seed = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
dataset_folder = '/Vols/vol_design/tools/swat/users/liord/datasets/mnist'
epochs = 2
gamma = 0.7
lr = 1.0

Let us define the dataset loaders, and optimizer and train the model for 2 epochs.

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST(dataset_folder, train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST(dataset_folder, train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, num_workers=1, pin_memory=True, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, num_workers=1, pin_memory=True,  batch_size=test_batch_size, shuffle=False)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.275661
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.270859
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.122138
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.129065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.147868
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.262936
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.105676
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.081128
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.091372
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.023000

Test set: Average loss: 0.0470, Accuracy: 9847/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.025655
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.012885
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.109077
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.019512
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.082994
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.042370
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.255850
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.184586
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.033581
T

After training for 2 epochs we get an accuracy of 98.84%. Not bad.

## Hardware-friendly quantization using MCT

Now we would like to quantize this model using the Model Compression Toolkit.
To do so, we need to define a representative dataset, which is a function that returns a list of images:

In [5]:
image_data_loader = iter(train_loader)

def representative_data_gen() -> list:
    return [next(image_data_loader)[0]]

Now for the fireworks. Lets run hardware-friendly post training quantization on the model. The output of MCT is a simulated quantized model in the input model's framework. That is, the model adds fake-quantization nodes after layers that need to be quantized. The output model's size on the disk does'nt change, but all the quantization parameters are available for deployment on target hardware.

In [6]:
quantized_model, quantization_info = mct.pytorch_post_training_quantization(
    model, 
    representative_data_gen, 
    n_iter=10)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.15it/s]


Approximated model size (in bytes): 1200584.0
Approximated compression ratio: 3.998


The MCT prints the approximated model size after real quantization and the compression ratio. In this example, we used the default setting of MCT and compressed the model from 32 bits to 8 bits, hence the compression ratio is x4. Using the simulated quantized model, we can evaluate its performance using the original model's testing environment, and compare its performance to the original model.

In [7]:
print(quantization_info)
test(quantized_model, device, test_loader)


Test set: Average loss: 0.0331, Accuracy: 9879/10000 (99%)



In this scenario, we see that the compression almost didn't affect the accuracy of the model. 

## Conclusion 

In this tutorial, we demonstrated how to quantize a classification model for MNIST in a hardware-friendly manner using MCT. We saw that we can achieve an x4 compression ratio with minimal performance degradation. 

The advantage of quantizing in a hardware-friendly manner is that this model can run more efficiently in the sense of run time, power consumption, and memory on designated hardware. 

This is a very simple model and a very simple task. MCT can demonstrate competitive results on a wide variety of tasks and network architectures. Check out the paper for more details: https://arxiv.org/abs/2109.09113